<a target="_blank" href="https://colab.research.google.com/github/instadeepai/jumanji/blob/main/examples/training.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [8]:
%%capture
!pip install --quiet -U "jumanji[train] @ git+https://github.com/instadeepai/jumanji.git@main"

In [9]:
# @title Set up JAX for available hardware (run me) { display-mode: "form" }

import subprocess
import os

# Based on https://stackoverflow.com/questions/67504079/how-to-check-if-an-nvidia-gpu-is-available-on-my-system
try:
    subprocess.check_output('nvidia-smi')
    print("a GPU is connected.")
except Exception:
    # TPU or CPU
    if "COLAB_TPU_ADDR" in os.environ and os.environ["COLAB_TPU_ADDR"]:
        import jax.tools.colab_tpu

        jax.tools.colab_tpu.setup_tpu()
        print("A TPU is connected.")
    else:
        print("Only CPU accelerator is connected.")

Only CPU accelerator is connected.


In [10]:
import warnings
warnings.filterwarnings("ignore")

from jumanji.training.train import train
from hydra import compose, initialize

In [11]:
env = "maze"  # @param ['bin_pack', 'cleaner', 'connector', 'cvrp', 'game_2048', 'graph_coloring', 'job_shop', 'knapsack', 'maze', 'minesweeper', 'mmst', 'multi_cvrp', 'robot_warehouse', 'rubiks_cube', 'snake', 'sudoku', 'tetris', 'tsp']
agent = "random"  # @param ['random', 'a2c']

In [12]:
#@title Download Jumanji Configs (run me) { display-mode: "form" }

import os
import requests


def download_file(url: str, file_path: str) -> None:
    # Send an HTTP GET request to the URL
    response = requests.get(url)
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        with open(file_path, "wb") as f:
            f.write(response.content)
    else:
        print("Failed to download the file.")


os.makedirs("configs", exist_ok=True)
config_url = "https://raw.githubusercontent.com/instadeepai/jumanji/main/jumanji/training/configs/config.yaml"
download_file(config_url, "configs/config.yaml")
env_url = f"https://raw.githubusercontent.com/instadeepai/jumanji/main/jumanji/training/configs/env/{env}.yaml"
os.makedirs("configs/env", exist_ok=True)
download_file(env_url, f"configs/env/{env}.yaml")

In [13]:
with initialize(version_base=None, config_path="configs"):
    cfg = compose(config_name="config.yaml", overrides=[f"env={env}", f"agent={agent}", "logger.type=terminal", "logger.save_checkpoint=true"])

train(cfg)

INFO:root:agent: random
seed: 0
logger:
  type: terminal
  save_checkpoint: true
  name: ${agent}_${env.name}
env:
  name: maze
  registered_version: Maze-v0
  network:
    num_channels:
    - 32
    - 32
    - 8
    policy_layers:
    - 64
    - 64
    value_layers:
    - 128
    - 128
  training:
    num_epochs: 100
    num_learner_steps_per_epoch: 500
    n_steps: 10
    total_batch_size: 128
  evaluation:
    eval_total_batch_size: 500
    greedy_eval_total_batch_size: 500
  a2c:
    normalize_advantage: false
    discount_factor: 0.99
    bootstrapping_factor: 0.95
    l_pg: 1.0
    l_td: 1.0
    l_en: 0.01
    learning_rate: 0.0003

INFO:root:{'devices': [CpuDevice(id=0)]}
INFO:root:Experiment: random_maze.
INFO:root:Starting logger.
INFO:root:Eval Stochastic >> Env Steps: 0.00e+00 | Episode Length: 80.542 | Episode Return: 0.294 | Time: 2.283
INFO:root:Train >> Env Steps: 0.00e+00 | Steps Per Second: 311,313 | Time: 2.056
INFO:root:Eval Stochastic >> Env Steps: 6.40e+05 | Episod